# Нейронная сеть Хопфилда. Пример распознавания изображений

## Математическое описание сети Хопфилда


### 1. Введение

Нейронная сеть Хопфилда — это рекуррентная ассоциативная память, способная восстанавливать и хранить устойчивые состояния (паттерны). Она представляет собой динамическую систему, эволюционирующую в сторону локального минимума энергетической функции.

### 2. Архитектура сети

Пусть сеть содержит $N$ нейронов. Состояние каждого нейрона обозначим как
[
x_i \in {-1, +1}, \quad i = 1, \dots, N.
]

Матрица весов имеет размерность $N \times N$:
[
W = (w_{ij}).
]

Сеть Хопфилда является полностью связанной, симметричной ($w_{ij} = w_{ji}$), без самосвязей:
[
w_{ii} = 0.
]


### 3. Обучение сети

Для запоминания $P$ бинарных паттернов $( { \xi^{(1)}, \xi^{(2)}, \dots, \xi^{(P)} } )$ используется правило Хопфилда:
$[
w_{ij} = \frac{1}{P} \sum_{p=1}^{P} \xi_i^{(p)} \xi_j^{(p)}, \quad i \neq j.
]
$
В матричной форме:
$[
W = \frac{1}{P} \sum_{p=1}^{P} \xi^{(p)} (\xi^{(p)})^T - P I.
]$


### 4. Процедура восстановления

Динамика сети задаётся обновлением состояний:
$[
x_i(t+1) = \text{sgn}\left( \sum_{j=1}^{N} w_{ij} x_j(t) \right).
]$

Обновление может быть:

* синхронным (все нейроны одновременно),
* асинхронным (по одному нейрону в случайном порядке).


### 5. Энергетическая функция

Каждому состоянию сети соответствует энергия:
$[
E(x) = -\frac{1}{2} \sum_{i \neq j} w_{ij} x_i x_j.
]$

Сеть эволюционирует так, что энергия никогда не возрастает:
$[
E(x(t+1)) \le E(x(t)).
]$

Локальные минимумы энергии — это запомненные паттерны.

### 6. Ёмкость сети

Максимальное число надёжно запоминаемых некоррелированных паттернов:
$[
P_{\max} \approx 0.138 N.
]$

При сильной корреляции между паттернами ёмкость резко падает.

## Установка необходимых библиотек и их импорт

In [173]:
!pip install numpy
!pip install Pillow
!pip install matplotlib
!pip install tensorflow
!pip install kagglehub

164169.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


164176.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


164181.76s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


164187.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


164193.02s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [174]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

##  Вспомогательные функции:
- функция для перевода черно-белого изображения в вектор состоящий из элементов $\{-1, +1\}$
- обратная функция для перевода вектора в соответсвующее изображение
- функция для добавления случаного шума (меняет случайные пиксели на противоположные в векторе, не в изначальном изображении)
  
  ---
###  Функция для сравнения:
- Метрика аккуратности (процент совпадающих пикселей)

In [185]:
def image_to_binary(image_path: str):
    img = Image.open(image_path)
    img = img.convert("RGBA")
    img = img.resize((28, 28))
    data = np.array(img)
    vector_image = [1 if any(j) else -1 for i in data for j in i]
    return vector_image


def binary_to_image(vector: list, size: int, filename: str, mode="L"):
    matrix = np.array(vector).reshape(size, size)
    
    if mode == "rgba":
        rgba = np.zeros((size, size, 4), dtype=np.uint8)
        rgba[matrix == 1] = [0, 0, 0, 255]
        rgba[matrix == -1] = [0, 0, 0, 0]
        Image.fromarray(rgba, mode='RGBA').save(filename)
        
    else:
        image_data = np.where(matrix == 1, 0, 255).astype(np.uint8)
        image = Image.fromarray(image_data, mode='L')
        image.save(filename)


def add_noise(pattern, noise=0.1):
    noisy = pattern.copy()
    n = len(pattern)
    num = int(n * noise)
    idx = np.random.choice(range(n), num, replace=False)
    for i in idx:
        noisy[i] *= -1
    return noisy


def recovery_accuracy(original, recovered):
    return np.mean(np.array(original) == np.array(recovered))

## Реализация модели нейронной сети Хопфилда

In [176]:
class HopfieldNetwork:
    def __init__(self, n_neurons):
        self.n = n_neurons
        self.W = np.zeros((n_neurons, n_neurons))
    
    def train(self, patterns):
        for p in patterns:
            self.W += np.outer(p, p)
        np.fill_diagonal(self.W, 0)
    
    def recall(self, pattern, steps=10):
        x = pattern.copy()
        for _ in range(steps):
            for i in range(self.n):
                s = np.dot(self.W[i], x)
                x[i] = 1 if s >= 0 else -1
        return x
    

## Обучение моделей

Обучим несколько моделей на разном количестве паттернов, чтобы далее сравнить их между собой

In [189]:
all_models = []
trains = list(map(str, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
for i in range(1, 10):
    data = trains[:i]
    ptrns = []
    for file in os.listdir(directory):
        filename = directory + "/" + file
        if ("_train.png" in filename) and (set(file) & set(data)):
            ptrns.append(image_to_binary(filename))
    mdl = HopfieldNetwork(len(ptrns[0]))
    mdl.train(ptrns)
    all_models.append(mdl)


## Сравнение моделей

Для начала посмотрим на картинку с шумом (слева оригинальное изображение, справа – с шумом в 10%)

In [186]:
zero_binary = image_to_binary("./MINIMNIST/0_train.png")
for noise_persent in range(0, 50, 5):
    noised_image = add_noise(zero_binary, noise_persent / 100)
    binary_to_image(noised_image, 28, f"./Images/0_noised_{noise_persent}.png", 'rgba')

<div style="display: flex; gap: 50px;">
  <img src="./MINIMNIST/0_train.png" width="100"/>
  <img src="./Images/0_noised_10.png" width="100"/>
</div>


Теперь у нас есть несколько испорченных изображений цифры 0, на которой обучались все модели. Теперь сравним их.
В качестве параметра по которому будем сравнивать выберем параметр "аккуратность" – этот параметр показывает процентное соотношение (в долях единицы) одинаковых пикселей у оригинального изображения и у восстановленного. Также пронаблюдаем за поведением модели при разном количестве итераций восстановления изображения, назовем количество шагов параметром epoch, тем больше этот параметр, тем больше итераций делает нейронная сеть для восстановления изображения

In [182]:
for epoch in range(1, 15):
    print(f'================================= epoch {epoch} =====================================')
    print("noise", end='\t')
    for i in range(len(all_models)):
        print(f'model_{i}', end='\t')
        
    print()
    
    for noise in range(0, 50, 5):
        i = 0 
        print(f'{noise}%', end='\t')
        for mdl in all_models:
            i += 1
            picture_noised = image_to_binary(f"./Images/0_noised_{noise}.png")
            restored_model = mdl.recall(picture_noised, epoch)
            persent = recovery_accuracy(zero_binary, restored_model)
            print(f'{persent:.5f}', end='\t')
        print()
    print()

================================= epoch 1 =====================================
noise	model_0	model_1	model_2	model_3	model_4	model_5	model_6	model_7	model_8	
0%	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	0.82270	
5%	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	0.89796	0.89668	0.80867	
10%	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	0.80867	
15%	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	0.82398	0.81250	0.78699	
20%	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	0.87117	0.86862	0.75765	
25%	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	0.89541	0.82781	0.74235	
30%	1.00000	1.00000	1.00000	1.00000	1.00000	1.00000	0.90944	0.90944	0.79847	
35%	1.00000	1.00000	1.00000	1.00000	1.00000	0.81888	0.74745	0.72959	0.68622	
40%	1.00000	1.00000	1.00000	0.70026	0.65944	0.70153	0.73469	0.73214	0.68495	
45%	1.00000	1.00000	0.99872	0.56378	0.56633	0.62755	0.71173	0.70918	0.64796	

================================= epoch 2 ==============================

## Интерпритация и анализ полученных результатов
#### Результаты
    1. Чем больше паттернов пытается запомнить модель, тем хуже результат
    
    2. Чем более шума присутствует на изображении, тем в среднем хуже модель
    восстанавливает это изображение
    
    3. При большом значении epoch результаты такие же, как и при более маленьких значениях
    
    4. Для моделей, обученных на 5 и более паттернах: чем больше количетсво итераций,
    тем хуже восстановленный результат

    5. Модель, обученная на всех паттернах не смогла распознать паттерн без шума
---
#### Анализ
    Для начала узнаем насколько похожи паттерны как векторы между собой:

In [188]:
coors = []
for num_1 in range(0, 10):
    filename_1 = f'./MINIMNIST/{num_1}_train.png'
    bin_1 = image_to_binary(filename_1)
    for num_2 in range(0, 10):
        if num_1 != num_2:
            filename_2 = f'./MINIMNIST/{num_2}_train.png'
            bin_2 = image_to_binary(filename_2)
            similar = recovery_accuracy(bin_1, bin_2)
            coors.append(similar)
print(f'Минимальная схожесть: {min(coors)}')
print(f'Максимальная схожесть: {max(coors)}')
print(f'Средняя схожесть: {sum(coors)/len(coors)}')

Минимальная схожесть: 0.4923469387755102
Максимальная схожесть: 0.7538265306122449
Средняя схожесть: 0.6160430839002268


Такие значения обуславливаются большим количетством пустого пространства (в нашем случае – прозрачного фона), а также похожестью написания некоторых цифр, например, таких как: $9$ $8$ $6$ 

Таким образом очевиден результат __1__, __3__, __4__: так как изначальные паттерны слишком сильно коррелируют между собой, то матрица W становится однообразной и вместо большого количества далеких энергетических минимумов мы наблюдаем за тем, как образуется один большой минимум и вокруг несколько маленьких. Из-за этого, даже когда на вход поступает исходное изображение model_8 не может его восстановить, а также при увеличении количества итераций модели 5, 6, 7 и 8 начинают хуже справляться с задачей, так как вместо того, чтобы распознать паттерн в "локальном минимуме" они скатываются в один глобальный энергетический минимум.

Результат 2 очевидный, так как чем дальше входное изображение находится от соответсвующего паттерна, следовательно ему "тяжелее" попасть в минимум соответствующий соответственному паттерну. Однако, когда модель обучена на небольшом количестве цифер, то она имеет несколько четких различных минимумов и достаточно быстро к ним сходится.

---
__Замечание__:
epoch ≠ количество повторений, которые проделывает модель для восстановления изображения. Она проделывает в $28^2$ раз большее количество итераций, это видно по исходному коду модели:  

```
def recall(self, pattern, steps=10):
    x = pattern.copy()
    for _ in range(steps):
        for i in range(self.n):
            s = np.dot(self.W[i], x)
            x[i] = 1 if s >= 0 else -1
    return 
```
        